# Build a Chatbot with Neural Network 

We've discovered how to build a chatbot with cosine similarity. Now, let's explore how we might build one with neural network!

We will create our training data, train a neural network with them, then use the trained model to make our chatbot. 

First, we will install required libraries. Uncomment the few blocks below only if you do not have the libraries installed. 

In [ ]:
#!pip install numpy scipy
#!pip install scikit-learn
#!pip install pillow
#!pip install h5py

In [ ]:
#!pip install tensorflow

In [ ]:
#!pip install tensorflow-gpu

In [ ]:
#!pip install keras

# 1. Install Libraries

Firstly, we will install libraries needed for this neural network powered chatbot. 
Keras is a machine learning library which utilizes tensorflow (another lower level machine learning library) at the backend. This makes it easier for us to deploy deep neural network for this purpose. 

In [1]:
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD
from keras.layers import Dense
 
from numpy import argmax
import numpy as np
import re

# 2. Input training data

We will first include the following training data for our chatbot:
1. X represent the different possible inputs that users might enter
2. Y represent the intent of the inputs

In [2]:
X = ['Hi',
     'Hello',
     'How are you?',
     'I am making',
     'making',
     'working',
     'studying',
     'see you later',
     'bye',
     'goodbye']

In [3]:
print(len(X))

10


In [4]:
Y = ['greeting',
     'greeting',
     'greeting',
     'busy',
     'busy',
     'busy',
     'busy',
     'bye',
     'bye',
     'bye']

In [5]:
print(len(Y))

10


Notice that there are several different sentences that have similar intent. Here, we are only having 3 intents, but you can add as many as you want for your project!

This is the way our chatbot will work:
1. From the input sentence, we will identify the intent using our trained AI model.
2. For each intent, we have a prepared response. 

For example, if we identify that the intent of the input is for a greeting, we might ask the chatbot to reply with a greeting as well, something like 'hi' or 'how are you doing?'

We will use machine learning to create a model that can classify input sentence into different intents. 
We make it as follows:

1. We create a training data (X and Y above) which contains a list of sentences and their intents.
2. Use the training data to train a classifier. 
3. Vectorize input sentences and use classifier to determine intent. 

# 3. Text processing

As usual, we will start with text processing. Do you remember the process?

## 3.1 Remove non alphanumeric characters

In [6]:
def remove_non_alpha_numeric_characters(sentence):
    new_sentence = ''
    for alphabet in sentence:
        if alphabet.isalpha() or alphabet == ' ':
            new_sentence += alphabet
    return new_sentence

In [7]:
def preprocess_data(X):
    X = [data_point.lower() for data_point in X]
    X = [remove_non_alpha_numeric_characters(
        sentence) for sentence in X]
    X = [data_point.strip() for data_point in X]
    X = [re.sub(' +', ' ',
                data_point) for data_point in X]
    return X

In [9]:
X = preprocess_data(X)

vocabulary = set()
for data_point in X:
    for word in data_point.split(' '):
        vocabulary.add(word)

vocabulary = list(vocabulary)

## Create document vectors

In [10]:
X_encoded = []

def encode_sentence(sentence):
    sentence = preprocess_data([sentence])[0]
    sentence_encoded = [0] * len(vocabulary)
    for i in range(len(vocabulary)):
        if vocabulary[i] in sentence.split(' '):
            sentence_encoded[i] = 1
    return sentence_encoded

X_encoded = [encode_sentence(sentence) for sentence in X]

In [11]:
classes = list(set(Y))

Y_encoded = []
for data_point in Y:
    data_point_encoded = [0] * len(classes)
    for i in range(len(classes)):
        if classes[i] == data_point:
            data_point_encoded[i] = 1
    Y_encoded.append(data_point_encoded)

# 4. Create training data and test data

In [14]:
X_train = X_encoded
y_train = Y_encoded
X_test = X_encoded
y_test = Y_encoded

Print and check the data you are using for training and test data

In [13]:
print (y_test)

[[0, 0, 1], [0, 0, 1], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0]]


In [19]:
print(X_train)

[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [ ]:
print(len(X_train))

10


In [ ]:
y_train

[[0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0]]

What does y_train represent? Do you understand the array shown above?

# 5. Model training

Now we will use the training data to train our neural network.

In [21]:
model = Sequential()
model.add(Dense(units=64, activation='sigmoid',
                input_dim=len(X_train[0])))
model.add(Dense(units=len(y_train[0]), activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 64)                960       
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 195       
Total params: 1,155
Trainable params: 1,155
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(loss=categorical_crossentropy,
              optimizer=SGD(lr=0.01,
                            momentum=0.9, nesterov=True))
model.fit(np.array(X_train), np.array(y_train), epochs=100, batch_size=16)

Epoch 1/100
1/1 [==============================] - 0s 273ms/step - loss: 0.8803
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 0.8798
Epoch 3/100
1/1 [==============================] - 0s 6ms/step - loss: 0.8792
Epoch 4/100
1/1 [==============================] - 0s 5ms/step - loss: 0.8783
Epoch 5/100
1/1 [==============================] - 0s 5ms/step - loss: 0.8773
Epoch 6/100
1/1 [==============================] - 0s 4ms/step - loss: 0.8761
Epoch 7/100
1/1 [==============================] - 0s 4ms/step - loss: 0.8748
Epoch 8/100
1/1 [==============================] - 0s 4ms/step - loss: 0.8733
Epoch 9/100
1/1 [==============================] - 0s 3ms/step - loss: 0.8718
Epoch 10/100
1/1 [==============================] - 0s 5ms/step - loss: 0.8702
Epoch 11/100
1/1 [==============================] - 0s 5ms/step - loss: 0.8684
Epoch 12/100
1/1 [==============================] - 0s 5ms/step - loss: 0.8666
Epoch 13/100
1/1 [==============================] - 0s 5ms/

## List down predictions

In [16]:
predictions = [argmax(pred) for pred in model.predict(np.array(X_test))]

# Model Evaluation

Let's evaluate our model now. We will compare the prediction made by the model and our test data:

In [17]:
correct = 0
for i in range(len(predictions)):
    if predictions[i] == argmax(y_test[i]):
        correct += 1

print ("Correct:", correct)
print ("Total:", len(predictions))

Correct: 8
Total: 10


# Testing the chatbot

Let's test the chatbot now! We will input a sentence, and then see what class is predicted by the neural network:

In [18]:
while True:
    print ("Enter a sentence")
    sentence = input()
    prediction= model.predict(np.array([encode_sentence(sentence)]))
    print (classes[argmax(prediction)])

Enter a sentence
hi
greeting
Enter a sentence
bye
busy
Enter a sentence


KeyboardInterrupt: ignored

Realize that you can't stop the chatbot? You'll have to add the exit command later (see the previous notebook to find out how to do it. 

For now, simply press the stop button (interrupt button) above to stop the chatbot. 

Try it! press the stop button, and try typing something onto the box. 

# Challenge

We have successfully use neural network to map our input to conversation intent. 
Your challenge is to link the conversation intent to a particular response that the chatbot will say. 
For example, if the conversation intent is 'greeting', get your chatbot to say a greeting as well!

In [ ]:
while True:
    print ("Enter a sentence")
    sentence = input()
    prediction= model.predict(np.array([encode_sentence(sentence)]))
    #print (classes[argmax(prediction)])
    if (classes[argmax(prediction)] == 'bye'):
        print ('Goodbye! See you next time!')
    elif (classes[argmax(prediction)] == 'busy'):
        print('ah... all the best!')
    elif (classes[argmax(prediction)] == 'greeting'):
        print ('Hi!')

Enter a sentence
hi
Hi!
Enter a sentence
busy
all the best!
Enter a sentence
bye
all the best!
Enter a sentence
goodbye
Goodbye! See you next time!
Enter a sentence
ciao
all the best!
Enter a sentence
osdagno
all the best!
Enter a sentence
you are fine man
Hi!
Enter a sentence


### Great job! You've successfully created a simple chatbot with neural network! How might you improve the chatbot?
You can improve the chatbot by:
- Adding more training data
- Adding more intent
- Focusing on a particular topic and train the chatbot with many training data in that topic

### Resource:
https://blog.eduonix.com/internet-of-things/simple-nlp-based-chatbot-python/